# Projet SDP - Explications pour la Somme Ponderee

## Contexte

Dans le concours de l'internat de medecine, les candidats sont classes selon une somme ponderee de leurs notes. L'objectif est de fournir une **explication comprehensible** de pourquoi un candidat x est mieux classe qu'un candidat y.

Au lieu de simplement montrer la formule de somme ponderee, on souhaite decomposer l'explication en **trade-offs** plus simples.

### Definitions generales

- **contribution[i]** = weight[i] × (x[i] - y[i])
- **pros(x,y)** = criteres ou x est meilleur que y (contribution > 0)
- **cons(x,y)** = criteres ou y est meilleur que x (contribution < 0)
- **neutral(x,y)** = criteres ou x et y sont egaux (contribution = 0)

In [162]:
# Imports
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

## Données du problème

In [163]:
# Donnees du probleme
criteria_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
full_names = ['Anatomie', 'Biologie', 'Chirurgie', 'Diagnostic', 
              'Epidemiologie', 'Forensic Pathology', 'Genetique']
weights = np.array([8, 7, 7, 6, 6, 5, 6])

# Tous les candidats
candidates = {
    'x': np.array([85, 81, 71, 69, 75, 81, 88]),
    'y': np.array([81, 81, 75, 63, 67, 88, 95]),
    'z': np.array([74, 89, 74, 81, 68, 84, 79]),
    't': np.array([74, 71, 84, 91, 77, 76, 73]),
    'u': np.array([72, 75, 66, 85, 88, 66, 93]),
    'v': np.array([71, 73, 63, 92, 76, 79, 93]),
    'w': np.array([79, 69, 78, 76, 67, 84, 79]),
    "w'": np.array([57, 76, 81, 76, 82, 86, 77]),
    'a1': np.array([89, 74, 81, 68, 84, 79, 77]),
    'a2': np.array([71, 84, 91, 79, 78, 73.5, 77])
}

# Afficher le tableau des candidats
print("Tableau des candidats:")
df_candidates = pd.DataFrame(candidates, index=criteria_names).T
df_candidates['Score'] = [np.sum(weights * candidates[c]) for c in candidates]
print(df_candidates.sort_values('Score', ascending=False))

print(f"\nPoids: {dict(zip(criteria_names, weights))}")

Tableau des candidats:
       A     B     C     D     E     F     G   Score
a1  89.0  74.0  81.0  68.0  84.0  79.0  77.0  3566.0
a2  71.0  84.0  91.0  79.0  78.0  73.5  77.0  3564.5
x   85.0  81.0  71.0  69.0  75.0  81.0  88.0  3541.0
y   81.0  81.0  75.0  63.0  67.0  88.0  95.0  3530.0
z   74.0  89.0  74.0  81.0  68.0  84.0  79.0  3521.0
t   74.0  71.0  84.0  91.0  77.0  76.0  73.0  3503.0
u   72.0  75.0  66.0  85.0  88.0  66.0  93.0  3489.0
v   71.0  73.0  63.0  92.0  76.0  79.0  93.0  3481.0
w   79.0  69.0  78.0  76.0  67.0  84.0  79.0  3413.0
w'  57.0  76.0  81.0  76.0  82.0  86.0  77.0  3395.0

Poids: {'A': 8, 'B': 7, 'C': 7, 'D': 6, 'E': 6, 'F': 5, 'G': 6}


## Fonctions utiles
Des fonctions utiles pour notre problème

In [164]:
def compute_contributions(x_scores, y_scores, weights):
    """Calcule les contributions de chaque critere a la somme ponderee dans x > y"""
    return weights * (x_scores - y_scores)

def classify_criteria(contributions, epsilon=1e-6):
    """Classifie les criteres en pros, cons et neutral"""
    pros = [i for i, c in enumerate(contributions) if c > epsilon]
    cons = [i for i, c in enumerate(contributions) if c < -epsilon]
    neutral = [i for i, c in enumerate(contributions) if abs(c) <= epsilon]
    return pros, cons, neutral

def display_comparison(cand1_name, cand2_name, candidates, weights, criteria_names):
    """Affiche les details d'une comparaison"""
    c1 = candidates[cand1_name]
    c2 = candidates[cand2_name]
    
    contributions = compute_contributions(c1, c2, weights)
    pros, cons, neutral = classify_criteria(contributions)
    
    print(f"Comparaison: {cand1_name} > {cand2_name}")
   
    
    df = pd.DataFrame({
        'Critere': criteria_names,
        'Poids': weights,
        cand1_name: c1,
        cand2_name: c2,
        'Contribution': contributions
    })
    print(df.to_string(index=False))
    
    score1 = np.sum(weights * c1)
    score2 = np.sum(weights * c2)
    print(f"\nScore {cand1_name} = {score1}")
    print(f"Score {cand2_name} = {score2}")
    print(f"Difference = {score1 - score2}")
    
    print(f"\npros({cand1_name},{cand2_name}) = {{{', '.join([criteria_names[i] for i in pros])}}}")
    print(f"cons({cand1_name},{cand2_name}) = {{{', '.join([criteria_names[i] for i in cons])}}}")
    print(f"neutral({cand1_name},{cand2_name}) = {{{', '.join([criteria_names[i] for i in neutral])}}}")
    
    return contributions, pros, cons, neutral

---
# Question 1 : Explication de type (1-1)

## Definition

- **Trade-off (1-1)** : paire (P, C) ou P ∈ pros, C ∈ cons, et contribution[P] + contribution[C] > 0
- **Explication (1-1)** : ensemble de trade-offs (1-1) disjoints couvrant tous les elements de cons(x,y)

## Formulation

### Variables
- z[p,c] ∈ {0,1} pour chaque paire valide (p ∈ pros, c ∈ cons)

### Contraintes
- C1: Chaque cons couvert exactement 1 fois: $\sum_p z_{p,c} = 1$
- C2: Chaque pro utilise au plus 1 fois: $\sum_c z_{p,c} \leq 1$
- C3: Variables creees uniquement pour paires valides (contribution[p] + contribution[c] > 0)

### Objectif
Minimiser le nombre de trade-offs: $\min \sum_{p,c} z_{p,c}$

In [165]:
def find_explanation_1_1(x_scores, y_scores, weights, criteria_names=None, verbose=True):
    """
    Question 1: Trouve une explication de type (1-1) pour x > y
    Trade-off (1-1): une paire (P, C) ou P compense C
    """
    n_criteria = len(x_scores)
    if criteria_names is None:
        criteria_names = [f"C{i}" for i in range(n_criteria)]
    
    contributions = compute_contributions(x_scores, y_scores, weights)
    pros, cons, neutral = classify_criteria(contributions)
    
    if verbose:
        print(f"\npros = {[criteria_names[i] for i in pros]}")
        print(f"cons = {[criteria_names[i] for i in cons]}")
    
    # Verifications
    if np.sum(contributions) <= 0:
        print("Erreur: x n'est pas meilleur que y")
        return None, "INVALID"
    
    if len(cons) == 0:
        if verbose:
            print("Aucun cons. Pas besoin d'explication.")
        return [], "TRIVIAL"
    
    # Identifier les paires valides
    epsilon = 0.01
    valid_pairs = []
    for p in pros:
        for c in cons:
            if contributions[p] + contributions[c] >= epsilon:
                valid_pairs.append((p, c))
    
    if verbose:
        print(f"\nPaires valides (1-1):")
        for p, c in valid_pairs:
            total = contributions[p] + contributions[c]
            print(f"  ({criteria_names[p]}, {criteria_names[c]}): {contributions[p]:+.1f} + ({contributions[c]:+.1f}) = {total:+.1f}")
    
    # Verifier que chaque cons peut etre couvert
    for c in cons:
        if not any(p for p in pros if (p, c) in valid_pairs):
            if verbose:
                print(f"\n=== Aucune explication (1-1) n'existe ===")
                print(f"Le cons {criteria_names[c]} ne peut etre couvert par aucun pro.")
            return None, "INFEASIBLE"
    
    # Modele Gurobi
    model = gp.Model("explanation_1_1")
    model.Params.OutputFlag = 0
    
    z = {}
    for p, c in valid_pairs:
        z[p, c] = model.addVar(vtype=GRB.BINARY, name=f"z_{criteria_names[p]}_{criteria_names[c]}")
    
    model.update()
    
    # C1: Chaque cons couvert exactement 1 fois
    for c in cons:
        model.addConstr(
            gp.quicksum(z[p, c] for p in pros if (p, c) in z) == 1,
            name=f"cover_{criteria_names[c]}"
        )
    
    # C2: Chaque pro utilise au plus 1 fois
    for p in pros:
        model.addConstr(
            gp.quicksum(z[p, c] for c in cons if (p, c) in z) <= 1,
            name=f"use_{criteria_names[p]}"
        )
    
    # Objectif
    model.setObjective(gp.quicksum(z[p, c] for p, c in valid_pairs), GRB.MINIMIZE)
    
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        explanation = [(p, c) for p, c in valid_pairs if z[p, c].X > 0.5]
        
        if verbose:
            print(f"\n=== Explication (1-1) trouvee ===")
            print(f"Longueur: {len(explanation)}")
            print("\nTrade-offs:")
            for p, c in explanation:
                total = contributions[p] + contributions[c]
                print(f"  ({criteria_names[p]}, {criteria_names[c]}): {contributions[p]:+.1f} + ({contributions[c]:+.1f}) = {total:+.1f} > 0")
            
            print("\nConclusion:")
            for p, c in explanation:
                print(f"  L'avantage en {criteria_names[p]} compense le desavantage en {criteria_names[c]}")
        
        return explanation, "OPTIMAL"
    
    elif model.status == GRB.INFEASIBLE:
        if verbose:
            print("\n Aucune explication (1-1) n'existe")
        return None, "INFEASIBLE"
    
    return None, f"STATUS_{model.status}"

### Test Question 1: x > y
On teste la question 1 pour le couple x et y, on s'attend à avoir une explication 

In [166]:
display_comparison('x', 'y', candidates, weights, criteria_names)
explanation, status = find_explanation_1_1(candidates['x'], candidates['y'], weights, criteria_names)

Comparaison: x > y
Critere  Poids  x  y  Contribution
      A      8 85 81            32
      B      7 81 81             0
      C      7 71 75           -28
      D      6 69 63            36
      E      6 75 67            48
      F      5 81 88           -35
      G      6 88 95           -42

Score x = 3541
Score y = 3530
Difference = 11

pros(x,y) = {A, D, E}
cons(x,y) = {C, F, G}
neutral(x,y) = {B}

pros = ['A', 'D', 'E']
cons = ['C', 'F', 'G']

Paires valides (1-1):
  (A, C): +32.0 + (-28.0) = +4.0
  (D, C): +36.0 + (-28.0) = +8.0
  (D, F): +36.0 + (-35.0) = +1.0
  (E, C): +48.0 + (-28.0) = +20.0
  (E, F): +48.0 + (-35.0) = +13.0
  (E, G): +48.0 + (-42.0) = +6.0

=== Explication (1-1) trouvee ===
Longueur: 3

Trade-offs:
  (A, C): +32.0 + (-28.0) = +4.0 > 0
  (D, F): +36.0 + (-35.0) = +1.0 > 0
  (E, G): +48.0 + (-42.0) = +6.0 > 0

Conclusion:
  L'avantage en A compense le desavantage en C
  L'avantage en D compense le desavantage en F
  L'avantage en E compense le desavantage 

### Test Question 1: w > w' 
Cette paire n'a pas d'explication possible 

In [167]:
display_comparison('w', "w'", candidates, weights, criteria_names)
explanation, status = find_explanation_1_1(candidates['w'], candidates["w'"], weights, criteria_names)

Comparaison: w > w'
Critere  Poids  w  w'  Contribution
      A      8 79  57           176
      B      7 69  76           -49
      C      7 78  81           -21
      D      6 76  76             0
      E      6 67  82           -90
      F      5 84  86           -10
      G      6 79  77            12

Score w = 3413
Score w' = 3395
Difference = 18

pros(w,w') = {A, G}
cons(w,w') = {B, C, E, F}
neutral(w,w') = {D}

pros = ['A', 'G']
cons = ['B', 'C', 'E', 'F']

Paires valides (1-1):
  (A, B): +176.0 + (-49.0) = +127.0
  (A, C): +176.0 + (-21.0) = +155.0
  (A, E): +176.0 + (-90.0) = +86.0
  (A, F): +176.0 + (-10.0) = +166.0
  (G, F): +12.0 + (-10.0) = +2.0

=== Aucune explication (1-1) n'existe ===
Le cons B ne peut etre couvert par aucun pro.


---
# Question 2 : Explication de type (1-m)

## Definition

- **Trade-off (1-m)** : paire (P, {C1, ..., Cm}) ou un seul pro P compense plusieurs cons
- Validite: contribution[P] + contribution[C1] + ... + contribution[Cm] > 0
- **Explication (1-m)** : ensemble de trade-offs (1-m) disjoints couvrant tous les cons

## Formulation

### Variables
- z[p,c] ∈ {0,1}: le cons c est assigne au pro p
- y[p] ∈ {0,1}: le pro p est utilise

### Contraintes
- C1: Chaque cons couvert exactement 1 fois
- C2: Lien z-y
- C3: Validite du trade-off pour chaque pro utilise

### Objectif
Minimiser le nombre de trade-offs (= nombre de pros utilises)

In [168]:
def find_explanation_1_m(x_scores, y_scores, weights, criteria_names=None, verbose=True):
    """
    Question 2: Trouve une explication de type (1-m) pour x > y
    Trade-off (1-m): un pro compense plusieurs cons
    """
    n_criteria = len(x_scores)
    if criteria_names is None:
        criteria_names = [f"C{i}" for i in range(n_criteria)]
    
    contributions = compute_contributions(x_scores, y_scores, weights)
    pros, cons, neutral = classify_criteria(contributions)
    
    if verbose:
        print(f"\npros = {[criteria_names[i] for i in pros]}")
        print(f"cons = {[criteria_names[i] for i in cons]}")
    
    if np.sum(contributions) <= 0:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"
    
    # Modele Gurobi
    model = gp.Model("explanation_1_m")
    model.Params.OutputFlag = 0
    
    z = {}
    for p in pros:
        for c in cons:
            z[p, c] = model.addVar(vtype=GRB.BINARY, name=f"z_{criteria_names[p]}_{criteria_names[c]}")
    
    y = {}
    for p in pros:
        y[p] = model.addVar(vtype=GRB.BINARY, name=f"y_{criteria_names[p]}")
    
    model.update()
    
    epsilon = 0.01
    M = 10000
    
    # C1: Chaque cons couvert exactement 1 fois
    for c in cons:
        model.addConstr(gp.quicksum(z[p, c] for p in pros) == 1, name=f"cover_{criteria_names[c]}")
    
    # C2: Lien z-y
    for p in pros:
        for c in cons:
            model.addConstr(z[p, c] <= y[p], name=f"link_{criteria_names[p]}_{criteria_names[c]}")
    
    # C3: Validite des trade-offs
    for p in pros:
        model.addConstr(
            contributions[p] + gp.quicksum(z[p, c] * contributions[c] for c in cons) >= epsilon - M * (1 - y[p]),
            name=f"valid_{criteria_names[p]}"
        )
    
    # Objectif
    model.setObjective(gp.quicksum(y[p] for p in pros), GRB.MINIMIZE)
    
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        explanation = []
        
        if verbose:
            print(f"\nExplication (1-m) trouvee")
            print(f"Longueur: {int(model.objVal)}")
            print("\nTrade-offs:")
        
        for p in pros:
            if y[p].X > 0.5:
                cons_for_p = [c for c in cons if z[p, c].X > 0.5]
                explanation.append((p, cons_for_p))
                
                if verbose:
                    cons_names = [criteria_names[c] for c in cons_for_p]
                    contrib_p = contributions[p]
                    contrib_cons = sum(contributions[c] for c in cons_for_p)
                    total = contrib_p + contrib_cons
                    print(f"  (1-{len(cons_for_p)}) ({criteria_names[p]}, {{{', '.join(cons_names)}}}): "
                          f"{contrib_p:+.1f} + ({contrib_cons:+.1f}) = {total:+.1f} > 0")
        
        if verbose:
            print("\n  Conclusion:")
            for p, cons_list in explanation:
                cons_names = [criteria_names[c] for c in cons_list]
                if len(cons_names) == 1:
                    print(f"  L'avantage en {criteria_names[p]} compense le desavantage en {cons_names[0]}")
                else:
                    print(f"  L'avantage en {criteria_names[p]} compense les desavantages en {', '.join(cons_names)}")
        
        return explanation, "OPTIMAL"
    
    elif model.status == GRB.INFEASIBLE:
        if verbose:
            print("\n Aucune explication (1-m) n'existe ")
        return None, "INFEASIBLE"
    
    return None, f"STATUS_{model.status}"

### Test Question 2: w > w' 
Cette paire est explicable en 1-m

In [169]:
display_comparison('w', "w'", candidates, weights, criteria_names)
explanation, status = find_explanation_1_m(candidates['w'], candidates["w'"], weights, criteria_names)

Comparaison: w > w'
Critere  Poids  w  w'  Contribution
      A      8 79  57           176
      B      7 69  76           -49
      C      7 78  81           -21
      D      6 76  76             0
      E      6 67  82           -90
      F      5 84  86           -10
      G      6 79  77            12

Score w = 3413
Score w' = 3395
Difference = 18

pros(w,w') = {A, G}
cons(w,w') = {B, C, E, F}
neutral(w,w') = {D}

pros = ['A', 'G']
cons = ['B', 'C', 'E', 'F']

Explication (1-m) trouvee
Longueur: 1

Trade-offs:
  (1-4) (A, {B, C, E, F}): +176.0 + (-170.0) = +6.0 > 0

  Conclusion:
  L'avantage en A compense les desavantages en B, C, E, F


### Test Question 2: u > v 
Cette paire doit échouer en l'explication 1-m

In [170]:
display_comparison('u', 'v', candidates, weights, criteria_names)
explanation, status = find_explanation_1_m(candidates['u'], candidates['v'], weights, criteria_names)

Comparaison: u > v
Critere  Poids  u  v  Contribution
      A      8 72 71             8
      B      7 75 73            14
      C      7 66 63            21
      D      6 85 92           -42
      E      6 88 76            72
      F      5 66 79           -65
      G      6 93 93             0

Score u = 3489
Score v = 3481
Difference = 8

pros(u,v) = {A, B, C, E}
cons(u,v) = {D, F}
neutral(u,v) = {G}

pros = ['A', 'B', 'C', 'E']
cons = ['D', 'F']

 Aucune explication (1-m) n'existe 


---
# Question 3 : Explication de type (m-1)

## Definition

- **Trade-off (m-1)** : paire ({P1, ..., Pm}, C) ou plusieurs pros compensent un seul cons
- Validite: contribution[P1] + ... + contribution[Pm] + contribution[C] > 0
- **Explication (m-1)** : ensemble de trade-offs (m-1) disjoints couvrant tous les cons

## Formulation

### Variables
- z[p,c] ∈ {0,1}: le pro p est assigne au cons c

### Contraintes
- C1: Chaque pro utilise au plus 1 fois
- C2: Validite du trade-off pour chaque cons

### Objectif
Minimiser le nombre de pros utilises

In [171]:
def find_explanation_m_1(x_scores, y_scores, weights, criteria_names=None, verbose=True):
    """
    Question 3: Trouve une explication de type (m-1) pour x > y
    Trade-off (m-1): plusieurs pros compensent un seul cons
    """
    n_criteria = len(x_scores)
    if criteria_names is None:
        criteria_names = [f"C{i}" for i in range(n_criteria)]
    
    contributions = compute_contributions(x_scores, y_scores, weights)
    pros, cons, neutral = classify_criteria(contributions)
    
    if verbose:
        print(f"\npros = {[criteria_names[i] for i in pros]}")
        print(f"cons = {[criteria_names[i] for i in cons]}")
    
    if np.sum(contributions) <= 0:
        return None, "INVALID"
    if len(cons) == 0:
        return [], "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"
    
    # Modele Gurobi
    model = gp.Model("explanation_m_1")
    model.Params.OutputFlag = 0
    
    z = {}
    for p in pros:
        for c in cons:
            z[p, c] = model.addVar(vtype=GRB.BINARY, name=f"z_{criteria_names[p]}_{criteria_names[c]}")
    
    model.update()
    
    epsilon = 0.01
    
    # C1: Chaque pro utilise au plus 1 fois
    for p in pros:
        model.addConstr(
            gp.quicksum(z[p, c] for c in cons) <= 1,
            name=f"use_{criteria_names[p]}"
        )
    
    # C2: Validite des trade-offs pour chaque cons
    for c in cons:
        model.addConstr(
            gp.quicksum(z[p, c] * contributions[p] for p in pros) + contributions[c] >= epsilon,
            name=f"valid_{criteria_names[c]}"
        )
    
    # Objectif
    model.setObjective(gp.quicksum(z[p, c] for p in pros for c in cons), GRB.MINIMIZE)
    
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        explanation = []
        
        if verbose:
            print(f"\nExplication (m-1) trouvee")
            print("\nTrade-offs:")
        
        for c in cons:
            pros_for_c = [p for p in pros if z[p, c].X > 0.5]
            explanation.append((pros_for_c, c))
            
            if verbose:
                pros_names = [criteria_names[p] for p in pros_for_c]
                contrib_pros = sum(contributions[p] for p in pros_for_c)
                contrib_c = contributions[c]
                total = contrib_pros + contrib_c
                print(f"  ({len(pros_for_c)}-1) ({{{', '.join(pros_names)}}}, {criteria_names[c]}): "
                      f"{contrib_pros:+.1f} + ({contrib_c:+.1f}) = {total:+.1f} > 0")
        
        if verbose:
            print(f"\nLongueur: {len(cons)} trade-offs")
            print("\nConclusion:")
            for pros_list, c in explanation:
                pros_names = [criteria_names[p] for p in pros_list]
                if len(pros_names) == 1:
                    print(f"  L'avantage en {pros_names[0]} compense le desavantage en {criteria_names[c]}")
                else:
                    print(f"  Les avantages en {', '.join(pros_names)} compensent le desavantage en {criteria_names[c]}")
        
        return explanation, "OPTIMAL"
    
    elif model.status == GRB.INFEASIBLE:
        if verbose:
            print("\n Aucune explication (m-1) n'existe")
        return None, "INFEASIBLE"
    
    return None, f"STATUS_{model.status}"

### Test Question 3: u > v 
On s'attend à ce qu'il y ait une explication de type m-1

In [172]:
display_comparison('u', 'v', candidates, weights, criteria_names)
explanation, status = find_explanation_m_1(candidates['u'], candidates['v'], weights, criteria_names)

Comparaison: u > v
Critere  Poids  u  v  Contribution
      A      8 72 71             8
      B      7 75 73            14
      C      7 66 63            21
      D      6 85 92           -42
      E      6 88 76            72
      F      5 66 79           -65
      G      6 93 93             0

Score u = 3489
Score v = 3481
Difference = 8

pros(u,v) = {A, B, C, E}
cons(u,v) = {D, F}
neutral(u,v) = {G}

pros = ['A', 'B', 'C', 'E']
cons = ['D', 'F']

Explication (m-1) trouvee

Trade-offs:
  (3-1) ({A, B, C}, D): +43.0 + (-42.0) = +1.0 > 0
  (1-1) ({E}, F): +72.0 + (-65.0) = +7.0 > 0

Longueur: 2 trade-offs

Conclusion:
  Les avantages en A, B, C compensent le desavantage en D
  L'avantage en E compense le desavantage en F


### Test Question 3: y > z

In [173]:
display_comparison('y', 'z', candidates, weights, criteria_names)
explanation, status = find_explanation_m_1(candidates['y'], candidates['z'], weights, criteria_names)

Comparaison: y > z
Critere  Poids  y  z  Contribution
      A      8 81 74            56
      B      7 81 89           -56
      C      7 75 74             7
      D      6 63 81          -108
      E      6 67 68            -6
      F      5 88 84            20
      G      6 95 79            96

Score y = 3530
Score z = 3521
Difference = 9

pros(y,z) = {A, C, F, G}
cons(y,z) = {B, D, E}
neutral(y,z) = {}

pros = ['A', 'C', 'F', 'G']
cons = ['B', 'D', 'E']

 Aucune explication (m-1) n'existe


### Test Question 3: z > t 
Pour ce cas ni 1-m ni m-1 ne fonctionne

In [174]:
display_comparison('z', 't', candidates, weights, criteria_names)
print("\n--- Test (1-m) ---")
exp_1m, status_1m = find_explanation_1_m(candidates['z'], candidates['t'], weights, criteria_names)
print("\n--- Test (m-1) ---")
exp_m1, status_m1 = find_explanation_m_1(candidates['z'], candidates['t'], weights, criteria_names)

Comparaison: z > t
Critere  Poids  z  t  Contribution
      A      8 74 74             0
      B      7 89 71           126
      C      7 74 84           -70
      D      6 81 91           -60
      E      6 68 77           -54
      F      5 84 76            40
      G      6 79 73            36

Score z = 3521
Score t = 3503
Difference = 18

pros(z,t) = {B, F, G}
cons(z,t) = {C, D, E}
neutral(z,t) = {A}

--- Test (1-m) ---

pros = ['B', 'F', 'G']
cons = ['C', 'D', 'E']

 Aucune explication (1-m) n'existe 

--- Test (m-1) ---

pros = ['B', 'F', 'G']
cons = ['C', 'D', 'E']

 Aucune explication (m-1) n'existe


---
# Question 4 : Explication combinee (1-m) et (m-1)

## Definition

Une explication combinee peut utiliser:
- Des trade-offs (1-m): un pro compense plusieurs cons
- Des trade-offs (m-1): plusieurs pros compensent un cons

## Strategie

On determine le **mode** de chaque cons:
- **'single'**: peut etre compense par 1 seul pro → eligible pour (1-m)
- **'multi'**: necessite plusieurs pros → doit utiliser (m-1)

## Formulation

### Variables
- z[p,c] ∈ {0,1}: le pro p contribue au cons c
- y[p] ∈ {0,1}: le pro p est utilise

### Contraintes
- Pour cons 'single': exactement 1 pro, avec validite globale du pro
- Pour cons 'multi': somme des contributions >= epsilon
- Exclusivite entre modes

In [ ]:
def find_explanation_combined(x_scores, y_scores, weights, criteria_names=None, verbose=True):
    """
    Question 4: Trouve une explication combinant (1-m) et (m-1) pour x > y
    
    Le mode de chaque cons (1-m ou m-1) est une VARIABLE DE DECISION,
    pas pre-determine. Le solveur choisit la meilleure configuration.
    """
    n_criteria = len(x_scores)
    if criteria_names is None:
        criteria_names = [f"C{i}" for i in range(n_criteria)]

    contributions = compute_contributions(x_scores, y_scores, weights)
    pros, cons, neutral = classify_criteria(contributions)

    if verbose:
        print(f"\npros = {[criteria_names[i] for i in pros]}")
        print(f"cons = {[criteria_names[i] for i in cons]}")
        print(f"\nContributions:")
        for i in pros:
            print(f"  {criteria_names[i]}: {contributions[i]:+.1f}")
        for i in cons:
            print(f"  {criteria_names[i]}: {contributions[i]:+.1f}")

    if np.sum(contributions) <= 0:
        print("Erreur: x n'est pas meilleur que y")
        return None, "INVALID"
    if len(cons) == 0:
        return {}, "TRIVIAL"
    if len(pros) == 0:
        return None, "INFEASIBLE"

    # Modele Gurobi
    model = gp.Model("explanation_combined")
    model.Params.OutputFlag = 0

    epsilon = 0.01
    M = 10000

    # === VARIABLES ===
    
    # z[p,c]: le pro p contribue au cons c
    z = {}
    for p in pros:
        for c in cons:
            z[p, c] = model.addVar(vtype=GRB.BINARY, name=f"z_{criteria_names[p]}_{criteria_names[c]}")

    # mode_1m[c]: 1 si cons c est en mode (1-m) (couvert par 1 seul pro)
    #             0 si cons c est en mode (m-1) (couvert par plusieurs pros)
    mode_1m = {}
    for c in cons:
        mode_1m[c] = model.addVar(vtype=GRB.BINARY, name=f"mode_{criteria_names[c]}")

    # y[p]: 1 si le pro p est utilise
    y = {}
    for p in pros:
        y[p] = model.addVar(vtype=GRB.BINARY, name=f"y_{criteria_names[p]}")

    model.update()

    # === CONTRAINTES ===

    # Lien y[p] avec z[p,c]
    for p in pros:
        for c in cons:
            model.addConstr(z[p, c] <= y[p], name=f"link_{criteria_names[p]}_{criteria_names[c]}")

    # Chaque cons doit etre couvert par au moins 1 pro
    for c in cons:
        model.addConstr(
            gp.quicksum(z[p, c] for p in pros) >= 1,
            name=f"cover_{criteria_names[c]}"
        )

    # Contrainte de mode: si mode_1m[c]=1, exactement 1 pro couvre c
    # si mode_1m[c]=0, au moins 1 pro (possiblement plusieurs)
    for c in cons:
        n_pros_c = gp.quicksum(z[p, c] for p in pros)
        # Si mode_1m = 1: n_pros_c = 1
        # Si mode_1m = 0: n_pros_c >= 1 (deja garanti par cover)
        model.addConstr(n_pros_c <= 1 + M * (1 - mode_1m[c]), name=f"mode_upper_{criteria_names[c]}")
        model.addConstr(n_pros_c >= 1 * mode_1m[c], name=f"mode_lower_{criteria_names[c]}")

    # Validite par cons (toujours): somme des contributions >= epsilon
    for c in cons:
        model.addConstr(
            gp.quicksum(z[p, c] * contributions[p] for p in pros) + contributions[c] >= epsilon,
            name=f"valid_cons_{criteria_names[c]}"
        )

    # Validite par pro (pour 1-m): si un pro couvre des cons en mode 1-m,
    # sa contribution + contributions de tous ses cons 1-m >= epsilon
    for p in pros:
        # Somme des contributions des cons en mode 1-m couverts par p
        # C'est: contrib[p] + sum_c (z[p,c] * mode_1m[c] * contrib[c])
        # Mais z[p,c] * mode_1m[c] est non-lineaire
        # On utilise une variable auxiliaire: w[p,c] = z[p,c] * mode_1m[c]
        pass  # On va simplifier ci-dessous

    # Simplification: on verifie la validite 1-m pour chaque pro
    # Si un pro p couvre plusieurs cons en mode 1-m, la somme doit etre valide
    # contrib[p] + sum_c(z[p,c] * contrib[c] pour c en mode 1-m) >= epsilon
    
    # Variable auxiliaire: w[p,c] = z[p,c] AND mode_1m[c]
    w = {}
    for p in pros:
        for c in cons:
            w[p, c] = model.addVar(vtype=GRB.BINARY, name=f"w_{criteria_names[p]}_{criteria_names[c]}")
            # w[p,c] = 1 ssi z[p,c]=1 ET mode_1m[c]=1
            model.addConstr(w[p, c] <= z[p, c])
            model.addConstr(w[p, c] <= mode_1m[c])
            model.addConstr(w[p, c] >= z[p, c] + mode_1m[c] - 1)

    model.update()

    # Validite 1-m pour chaque pro: si le pro est utilise en mode 1-m
    for p in pros:
        # has_1m[p] = 1 si p couvre au moins un cons en mode 1-m
        has_1m = model.addVar(vtype=GRB.BINARY, name=f"has1m_{criteria_names[p]}")
        sum_w = gp.quicksum(w[p, c] for c in cons)
        model.addConstr(has_1m <= sum_w)
        model.addConstr(has_1m * M >= sum_w)
        
        # Si has_1m = 1, alors contrib[p] + sum(w[p,c] * contrib[c]) >= epsilon
        model.addConstr(
            contributions[p] + gp.quicksum(w[p, c] * contributions[c] for c in cons) >= epsilon - M * (1 - has_1m),
            name=f"valid_pro_{criteria_names[p]}"
        )

    # Exclusivite: un pro utilise en mode (m-1) ne peut pas couvrir d'autres cons
    # Si z[p,c]=1 et mode_1m[c]=0, alors p ne couvre que c
    for p in pros:
        for c in cons:
            # z[p,c]=1 et mode_1m[c]=0 implique sum_{c'!=c} z[p,c'] = 0
            # Equivalent: sum_{c'} z[p,c'] <= 1 + M*(1-z[p,c]) + M*mode_1m[c]
            model.addConstr(
                gp.quicksum(z[p, c2] for c2 in cons) <= 1 + M * (1 - z[p, c]) + M * mode_1m[c],
                name=f"excl_{criteria_names[p]}_{criteria_names[c]}"
            )

    # === OBJECTIF ===
    model.setObjective(gp.quicksum(z[p, c] for p in pros for c in cons), GRB.MINIMIZE)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        if verbose:
            print(f"\n=== Explication combinee trouvee ===")
            print(f"Nombre de liens: {int(model.objVal)}")
        
        # Construire l'explication
        tradeoffs_1m = []  # (pro, [cons])
        tradeoffs_m1 = []  # ([pros], cons)
        
        # Identifier les modes
        cons_in_1m = [c for c in cons if mode_1m[c].X > 0.5]
        cons_in_m1 = [c for c in cons if mode_1m[c].X < 0.5]
        
        # Trade-offs (1-m): regrouper les cons par pro
        pro_to_cons = {}
        for c in cons_in_1m:
            for p in pros:
                if z[p, c].X > 0.5:
                    if p not in pro_to_cons:
                        pro_to_cons[p] = []
                    pro_to_cons[p].append(c)
        
        for p, cons_list in pro_to_cons.items():
            tradeoffs_1m.append((p, cons_list))
        
        # Trade-offs (m-1): pour chaque cons en mode m-1
        for c in cons_in_m1:
            pros_for_c = [p for p in pros if z[p, c].X > 0.5]
            tradeoffs_m1.append((pros_for_c, c))
        
        if verbose:
            print("\nTrade-offs:")
            
            for p, cons_list in tradeoffs_1m:
                cons_names = [criteria_names[c] for c in cons_list]
                contrib_p = contributions[p]
                contrib_cons = sum(contributions[c] for c in cons_list)
                total = contrib_p + contrib_cons
                print(f"  (1-{len(cons_list)}) ({criteria_names[p]}, {{{', '.join(cons_names)}}}): "
                      f"{contrib_p:+.1f} + ({contrib_cons:+.1f}) = {total:+.1f} > 0")
            
            for pros_list, c in tradeoffs_m1:
                pros_names = [criteria_names[p] for p in pros_list]
                contrib_pros = sum(contributions[p] for p in pros_list)
                contrib_c = contributions[c]
                total = contrib_pros + contrib_c
                print(f"  ({len(pros_list)}-1) ({{{', '.join(pros_names)}}}, {criteria_names[c]}): "
                      f"{contrib_pros:+.1f} + ({contrib_c:+.1f}) = {total:+.1f} > 0")

            print("\nConclusion:")
            for p, cons_list in tradeoffs_1m:
                cons_names = [criteria_names[c] for c in cons_list]
                if len(cons_names) == 1:
                    print(f"  L'avantage en {criteria_names[p]} compense le desavantage en {cons_names[0]}")
                else:
                    print(f"  L'avantage en {criteria_names[p]} compense les desavantages en {', '.join(cons_names)}")
            
            for pros_list, c in tradeoffs_m1:
                pros_names = [criteria_names[p] for p in pros_list]
                if len(pros_names) == 1:
                    print(f"  L'avantage en {pros_names[0]} compense le desavantage en {criteria_names[c]}")
                else:
                    print(f"  Les avantages en {', '.join(pros_names)} compensent le desavantage en {criteria_names[c]}")

        return {'1m': tradeoffs_1m, 'm1': tradeoffs_m1}, "OPTIMAL"

    elif model.status == GRB.INFEASIBLE:
        if verbose:
            print("\n Aucune explication combinee n'existe ")
        return None, "INFEASIBLE"

    return None, f"STATUS_{model.status}"

### Test Question 4: z > t (necessite combinaison)

In [176]:
display_comparison('z', 't', candidates, weights, criteria_names)
explanation, status = find_explanation_combined(candidates['z'], candidates['t'], weights, criteria_names)

Comparaison: z > t
Critere  Poids  z  t  Contribution
      A      8 74 74             0
      B      7 89 71           126
      C      7 74 84           -70
      D      6 81 91           -60
      E      6 68 77           -54
      F      5 84 76            40
      G      6 79 73            36

Score z = 3521
Score t = 3503
Difference = 18

pros(z,t) = {B, F, G}
cons(z,t) = {C, D, E}
neutral(z,t) = {A}

pros = ['B', 'F', 'G']
cons = ['C', 'D', 'E']

Contributions:
  B: +126.0
  F: +40.0
  G: +36.0
  C: -70.0
  D: -60.0
  E: -54.0

=== Explication combinee trouvee ===
Nombre de liens: 4

Trade-offs:
  (1-2) (B, {D, E}): +126.0 + (-114.0) = +12.0 > 0
  (2-1) ({F, G}, C): +76.0 + (-70.0) = +6.0 > 0

En langage naturel:
  L'avantage en B compense les desavantages en D, E
  Les avantages en F, G compensent le desavantage en C


### Test Question 4: a1 > a2

In [177]:
display_comparison('a1', 'a2', candidates, weights, criteria_names)
explanation, status = find_explanation_combined(candidates['a1'], candidates['a2'], weights, criteria_names)

Comparaison: a1 > a2
Critere  Poids  a1   a2  Contribution
      A      8  89 71.0         144.0
      B      7  74 84.0         -70.0
      C      7  81 91.0         -70.0
      D      6  68 79.0         -66.0
      E      6  84 78.0          36.0
      F      5  79 73.5          27.5
      G      6  77 77.0           0.0

Score a1 = 3566
Score a2 = 3564.5
Difference = 1.5

pros(a1,a2) = {A, E, F}
cons(a1,a2) = {B, C, D}
neutral(a1,a2) = {G}

pros = ['A', 'E', 'F']
cons = ['B', 'C', 'D']

Contributions:
  A: +144.0
  E: +36.0
  F: +27.5
  B: -70.0
  C: -70.0
  D: -66.0

 Aucune explication combinee n'existe 


---
# Resume: Toutes les comparaisons du classement

In [178]:
# Classement: x > y > z > t > u > v > w > w'
ranking = ['x', 'y', 'z', 't', 'u', 'v', 'w', "w'"]

print("="*80)
print("RESUME: Types d'explications pour chaque comparaison")
print("="*80)

results = []

for i in range(len(ranking) - 1):
    c1, c2 = ranking[i], ranking[i+1]
    
    # Test (1-1)
    _, status_11 = find_explanation_1_1(candidates[c1], candidates[c2], weights, criteria_names, verbose=False)
    # Test (1-m)
    _, status_1m = find_explanation_1_m(candidates[c1], candidates[c2], weights, criteria_names, verbose=False)
    # Test (m-1)
    _, status_m1 = find_explanation_m_1(candidates[c1], candidates[c2], weights, criteria_names, verbose=False)
    # Test combine
    _, status_comb = find_explanation_combined(candidates[c1], candidates[c2], weights, criteria_names, verbose=False)
    
    results.append({
        'Comparaison': f"{c1} > {c2}",
        '(1-1)': 'OK' if status_11 == 'OPTIMAL' else '-',
        '(1-m)': 'OK' if status_1m == 'OPTIMAL' else '-',
        '(m-1)': 'OK' if status_m1 == 'OPTIMAL' else '-',
        'Combinee': 'OK' if status_comb == 'OPTIMAL' else '-'
    })

df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

RESUME: Types d'explications pour chaque comparaison
Comparaison (1-1) (1-m) (m-1) Combinee
      x > y    OK    OK    OK       OK
      y > z     -     -     -        -
      z > t     -     -     -       OK
      t > u     -     -     -        -
      u > v     -     -    OK       OK
      v > w     -     -    OK       OK
     w > w'     -    OK     -       OK


---
# Fonction utilitaire: Expliquer une comparaison

In [179]:
def explain(cand1_name, cand2_name):
    """
    Fonction complete pour expliquer pourquoi cand1 > cand2
    Essaie tous les types d'explications et retourne la plus simple
    """
    c1 = candidates[cand1_name]
    c2 = candidates[cand2_name]
    
    display_comparison(cand1_name, cand2_name, candidates, weights, criteria_names)
    
    print("\n" + "="*60)
    print("Recherche de la meilleure explication...")
    print("="*60)
    
    # Essayer (1-1)
    exp, status = find_explanation_1_1(c1, c2, weights, criteria_names, verbose=False)
    if status == "OPTIMAL":
        print("\n>>> Explication (1-1) trouvee!")
        find_explanation_1_1(c1, c2, weights, criteria_names, verbose=True)
        return
    
    # Essayer (1-m)
    exp, status = find_explanation_1_m(c1, c2, weights, criteria_names, verbose=False)
    if status == "OPTIMAL":
        print("\n>>> Explication (1-m) trouvee!")
        find_explanation_1_m(c1, c2, weights, criteria_names, verbose=True)
        return
    
    # Essayer (m-1)
    exp, status = find_explanation_m_1(c1, c2, weights, criteria_names, verbose=False)
    if status == "OPTIMAL":
        print("\n>>> Explication (m-1) trouvee!")
        find_explanation_m_1(c1, c2, weights, criteria_names, verbose=True)
        return
    
    # Essayer combinee
    exp, status = find_explanation_combined(c1, c2, weights, criteria_names, verbose=False)
    if status == "OPTIMAL":
        print("\n>>> Explication combinee trouvee!")
        find_explanation_combined(c1, c2, weights, criteria_names, verbose=True)
        return
    
    print("\n>>> Aucune explication trouvee!")

In [180]:
# Exemple d'utilisation
explain('z', 't')

Comparaison: z > t
Critere  Poids  z  t  Contribution
      A      8 74 74             0
      B      7 89 71           126
      C      7 74 84           -70
      D      6 81 91           -60
      E      6 68 77           -54
      F      5 84 76            40
      G      6 79 73            36

Score z = 3521
Score t = 3503
Difference = 18

pros(z,t) = {B, F, G}
cons(z,t) = {C, D, E}
neutral(z,t) = {A}

Recherche de la meilleure explication...

>>> Explication combinee trouvee!

pros = ['B', 'F', 'G']
cons = ['C', 'D', 'E']

Contributions:
  B: +126.0
  F: +40.0
  G: +36.0
  C: -70.0
  D: -60.0
  E: -54.0

=== Explication combinee trouvee ===
Nombre de liens: 4

Trade-offs:
  (1-2) (B, {D, E}): +126.0 + (-114.0) = +12.0 > 0
  (2-1) ({F, G}, C): +76.0 + (-70.0) = +6.0 > 0

En langage naturel:
  L'avantage en B compense les desavantages en D, E
  Les avantages en F, G compensent le desavantage en C


In [181]:
explain('a1', 'a2')

Comparaison: a1 > a2
Critere  Poids  a1   a2  Contribution
      A      8  89 71.0         144.0
      B      7  74 84.0         -70.0
      C      7  81 91.0         -70.0
      D      6  68 79.0         -66.0
      E      6  84 78.0          36.0
      F      5  79 73.5          27.5
      G      6  77 77.0           0.0

Score a1 = 3566
Score a2 = 3564.5
Difference = 1.5

pros(a1,a2) = {A, E, F}
cons(a1,a2) = {B, C, D}
neutral(a1,a2) = {G}

Recherche de la meilleure explication...

>>> Aucune explication trouvee!
